### This script file is used to analysis Default.csv dataset.

<b>We want to use logistic regression to predict the probability of default using income and balance on the Default data set. Our goal is to estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.</b>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/Default.csv')
df = df.drop('index', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
default    10000 non-null object
student    10000 non-null object
balance    10000 non-null float64
income     10000 non-null float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [3]:
description = df.describe()
print(description)

            balance        income
count  10000.000000  10000.000000
mean     835.374886  33516.981876
std      483.714985  13336.639563
min        0.000000    771.967729
25%      481.731105  21340.462905
50%      823.636973  34552.644800
75%     1166.308387  43807.729275
max     2654.322576  73554.233500


<b>(a) Fit a logistic regression model that uses income and balance to predict default.</b>

<b>(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:
1. Split the sample set into a training set and a validation set.
1. Fit a multiple logistic regression model using only the training observations.
1. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.
1. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.</b>

<b>(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Describe your findings and comment on the results obtained.</b>

<b>(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.</b>